<a href="https://colab.research.google.com/github/aydanali/ECON3916-Stats-and-ML/blob/main/Lab6/Lab6_StatsML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import seaborn as sns
import pandas as pd
import numpy as np

# 1. Data Ingestion (The Population)
df = sns.load_dataset('titanic')
print(f"Total Population: {len(df)}")
print(f"Population Survival Rate: {df['survived'].mean():.4f}")

# 2. Manual Shuffle (Simulation of Sampling)
# We set a seed to ensure reproducibility for the lesson,
# but in production, this variance happens naturally.
np.random.seed(2026)
indices = np.random.permutation(df.index)

Total Population: 891
Population Survival Rate: 0.3838


In [5]:
# 3. Cut the deck (80/20 Split)
split_point = int(len(df) * 0.8)

# Slicing the shuffled indices
train_idx = indices[:split_point]
test_idx = indices[split_point:]

# Creating the subsets
train_set = df.loc[train_idx]
test_set = df.loc[test_idx]

# 4. Bias Check (The Delta)
train_surv = train_set['survived'].mean()
test_surv = test_set['survived'].mean()
delta = abs(train_surv - test_surv)

print(f"Train Survival Rate: {train_surv:.4f}")
print(f"Test Survival Rate:  {test_surv:.4f}")
print(f"Sampling Bias (Delta): {delta:.4f}")

Train Survival Rate: 0.3736
Test Survival Rate:  0.4246
Sampling Bias (Delta): 0.0510


In [7]:
from sklearn.model_selection import train_test_split

# Stratify by 'pclass' ensures the distribution of classes is identical
X_train, X_test = train_test_split(df, test_size=0.2, stratify=df['pclass'],
                                   random_state=42)

print("\n--- Stratified Split ---")
print("Train Class Dist:\n", X_train['pclass'].value_counts(normalize=True))
print("Test Class Dist:\n", X_test['pclass'].value_counts(normalize=True))


--- Stratified Split ---
Train Class Dist:
 pclass
3    0.558989
1    0.228933
2    0.212079
Name: proportion, dtype: float64
Test Class Dist:
 pclass
3    0.519553
1    0.296089
2    0.184358
Name: proportion, dtype: float64


In [8]:
from scipy.stats import chisquare

# Define observed and expected frequencies
observed = [450, 550]  # Control, Treatment
expected = [500, 500]  # 50/50 split for 1000 users

# Perform Chi-Square test
chi2_stat, p_value = chisquare(f_obs=observed, f_exp=expected)

# Print results
print(f"Chi-Square Statistic: {chi2_stat:.4f}")
print(f"P-value: {p_value:.6f}")
print()

# Conclusion
if p_value < 0.01:
    print("CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.")
else:
    print("Variance is within natural limits.")

Chi-Square Statistic: 10.0000
P-value: 0.001565

CRITICAL FAILURE: Sample Ratio Mismatch (SRM) Detected. Check Load Balancer.
